# Black-Litterman Portfolio's Theory #

### Practical Example ###

In [2]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from functions import import_financial_data
from portfolios_toolkit import calculate_analytics

In [3]:
# Import Data

# Apple Data
df_1 = import_financial_data("AAPL", mkt_cap=True)

# Amazon Data
df_2 =  import_financial_data("AMZN", mkt_cap=True)

# Meta Data
df_3 =  import_financial_data("META", mkt_cap=True)

# Microsoft Data
df_4 =  import_financial_data("MSFT", mkt_cap=True)

In [4]:
# Create the joint dataframe
df_data = pd.DataFrame()

# Columns will be the Adjusted Close Price
df_data['AAPL'] = df_1['adj_close']
df_data['AMZN'] = df_2['adj_close']
df_data['META'] = df_3['adj_close']
df_data['MSFT'] = df_4['adj_close']

# Drop Missing Data
df_data = df_data.dropna()

df_returns = df_data.pct_change(1).mul(100)
df_returns = df_returns.apply(lambda x: x.fillna(x.mean()), axis=0)

df_returns

In [5]:
# Theoretically we could use the average as the expected returns (these are daily returns)
expected_returns = df_returns.mean() 

expected_returns

In [6]:
# The volatility is calculated with the standard deviations
volatility = df_returns.dropna().std()

volatility

In [7]:
# Covariance Matrix
cov_matrix = df_returns.dropna().cov()

cov_matrix

In [9]:
# Create the Market Capitalization 
df_mktcap = pd.DataFrame()

df_mktcap['AAPL'] = np.sqrt(df_1['mkt_cap'])
df_mktcap['AMZN'] = np.sqrt(df_2['mkt_cap'])
df_mktcap['META'] = np.sqrt(df_3['mkt_cap'])
df_mktcap['MSFT'] = np.sqrt(df_4['mkt_cap'])

df_mktcap = df_mktcap.dropna()

df_mktcap

In [10]:
# Create the Market Capitalization Weights

total_market_cap = df_mktcap.sum(axis=1)  # Horizontal Sum because we got a Time Series

df_mktcap_weights = df_mktcap.div(total_market_cap, axis = 0)

df_mktcap_weights

In [11]:
# Let us use the last observations since the mean of the portfolio is calculated with the whole story

market_weights = df_mktcap_weights.iloc[-1]

market_weights

In [12]:
# We can calculate or estimate the Risk Aversion Coefficient using market data, but let us assume it

risk_aversion = 2.5

In [13]:
# Compute implied equilibrium returns

pi = risk_aversion * cov_matrix @ market_weights

pi

In [14]:
# Tau adjustment (controls uncertainty of implied returns)
tau = 0.10

pi_adjusted = tau * pi

pi_adjusted

In [15]:
# P matrix: 1 view per asset (identity matrix)
P = np.identity(4)

# Q vector: our expected returns relative to the market
Q = np.array([0.01, 0.02, 0.05, 0.03])  # AAPL, AMZN, META, MSFT

In [16]:
# Diagonal Omega matrix: uncertainty of each view
# Often proportional to variance of the assets related to the views
Omega = np.diag(np.diag(P @ cov_matrix @ P.T)) * tau

Omega

In [17]:
# Inverse Matrix

inv_tau_sigma = np.linalg.inv(tau * cov_matrix)
inv_omega = np.linalg.inv(Omega)

In [18]:
# Apply Black-Litterman formula

posterior_mean = np.linalg.inv(inv_tau_sigma + P.T @ inv_omega @ P) @ (inv_tau_sigma @ pi + P.T @ inv_omega @ Q)

posterior_mean

In [19]:
# Calculate the Optimal Weights

optimal_weights = np.linalg.inv(cov_matrix) @ (posterior_mean) / risk_aversion

# Normalize weights to sum to 1

optimal_weights /= np.sum(optimal_weights)

optimal_weights

In [20]:
# Calculate the Portfolio Returns and Variance

portfolio_returns = np.dot(optimal_weights, expected_returns)

portfolio_variance = np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights))

portfolio_volatility = np.sqrt(portfolio_variance)

print(f"Expected Portfolio Return: {portfolio_returns:.4f}")
print(f"Portfolio Variance: {portfolio_variance:.6f}")
print(f"Portfolio Volatility (Std Dev): {portfolio_volatility:.4f}")

In [23]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(df_returns.cumsum(), label=df_returns.columns, alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('$r_t$')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [24]:
# Calculate the Historical Returns of the Portfolio

portfolio_returns = df_returns @ optimal_weights

portfolio_returns

In [25]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(df_returns.cumsum(), label=df_returns.columns, alpha=1)
plt.plot(portfolio_returns.cumsum(), label='BL Portfolio', color='purple', alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('Cumulative Returns')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [26]:
# Create the Analytics

df_returns['BL'] = portfolio_returns

df_returns

In [27]:
# Now the table
analytics_table = calculate_analytics(df_returns)

analytics_table